# Uporaba tabel 1-N

Zadnjič smo v podatkovje dodajali stolpce, katerih vrednosti so bile določene z natanko eno vrednostjo v eni od tabel `hours` ali `attributes`.

Danes si bomo ogledali, kako to storimo za stolpce, katerih vrednost temelji na poljubnem številu vrednosti v eni od tabel.

Primer takega stolpca je npr. uporaba stolpca `stars` iz tabele `reviews`, saj vsakemu poslovnemu obratu pripada 0, 1 ali več recenzij.

![shema podatkovja Yelp](tex/shema.png)

In [6]:
# nekaj podporne kode

import itertools
import pandas as pd
import numpy as np
from typing import List, Union
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


def preberi_vse(tabele):
    """
    Naloži vse podatke
    :return: slovar {ime tabele: pripadajoč pd.DataFrame}
    """
    #ta_dir = os.path.dirname(os.path.abspath(__file__))
    #return {tabela: pd.read_csv(os.path.join(ta_dir, f"yelp_{tabela}.txt"), sep="\t") for tabela in tabele}
    return {tabela: pd.read_csv(f"yelp_{tabela}.txt", sep="\t") for tabela in tabele}


def pretvori_vse_one_hot(df: pd.DataFrame, stolpci: List[str], drop: Union[str, List[str]] = "first"):
    """
    Pretvori vsak kategoričen (= nominalen) atribut v nekaj 0/1-atributov (0/1-kodiranje).

    :param df: pandas tabela podatkov
    :param stolpci: seznam imen vhodnih spremenljivk
    :param drop: taktika odmeta za enega od ustvarjenih 0/1-atributov (1 je odveč ...)
    :return: pretvorjen df
    """
    bloki = []
    for stolpec in stolpci:
        x = np.array(df[stolpec]).reshape((-1, 1))
        tip = x.dtype.name
        if "float" not in tip and "int" not in tip:
            encoder = OneHotEncoder(drop=drop, sparse_output=False)
            x_v_numericni_obliki = encoder.fit_transform(x)
        else:
            x_v_numericni_obliki = x
        bloki.append(x_v_numericni_obliki)
    return np.block(bloki)


def preizkusi_podatke(xs: np.ndarray, y: np.ndarray):
    """
    Razdeli tabelo xs in vektor y na učni (75%) in testni (25%) del ter preveri, kako dobro se da napovedati
    vrednosti ciljne spremenljivke z naključnim gozdom.

    Za primerjavo naredi enako še z modelom, ki vedno napove večinski razred.

    Izpiše izmerjeni točnosti.

    :param xs: 2D tabela vrednosti vhodnih spremenljivk
    :param y: 1D tabela vrednosti ciljne spremenljivke
    :return: None
    """
    x_ucna, x_testna, y_ucna, y_testna = train_test_split(xs, y, test_size=0.25, random_state=1234, stratify=y)
    modeli = [
        ("Naključni gozd", RandomForestClassifier(random_state=1234)),
        ("Bučman", DummyClassifier())
    ]
    rezultati = {}
    for ime, model in modeli:
        model.fit(x_ucna, y_ucna)
        napovedi = model.predict(x_testna)
        rezultati[ime] = accuracy_score(y_testna, napovedi)
    return rezultati


def generiraj_vse_neprazne_podsezname(n):
    """
    Generira vse neprazne podsezname seznama z n elementi.
    Namesto dejanskih elementov le indekse.
    :param n:
    :return:
    """
    podseznami = []
    for velikost in range(1, n + 1):
        for moznost in itertools.combinations(range(n), velikost):
            podseznami.append(moznost)
    return podseznami


## Nekaj globalnih spremenljivk

In [7]:
TABELE = [
    "attributes",
    "business",
    "hours",
    "review",
    "users"
]
PODATKI = preberi_vse(TABELE)

business = PODATKI["business"]
biznisi = list(business["business_id"])  # določa vrstni red: uporabi za razširitve

stolpci = list(business.columns)
stolpci_x = stolpci[3:-1]

xs_business = pretvori_vse_one_hot(business, stolpci_x)
y_business = np.array(business[stolpci[-1]])

# Razširitvi od zadnjič

In [8]:
def razsiri_z_hours(business_ids):
    """
    Naredimo 14 novih atributov: PON open, PON close, TOR open, TOR close, ...

    """
    dnevi = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    stolpci = [f"{dan} {kaj}" for dan in dnevi for kaj in ["open", "close"]]
    hours_spremenjeno = pd.DataFrame(index=business_ids, columns=stolpci, data=30)

    hours_original = PODATKI["hours"].copy()
    # spremenimo vrednosti
    for stolp in ["open time", "close time"]:
        hours_original[stolp] = hours_original[stolp].apply(lambda cas: int(cas[:cas.find(":")]))  # "11:00:00" --> 11

    for _, vrsta in hours_original.iterrows():
        business_id = vrsta["business_id"]
        dan = vrsta["day"]
        odpremo = vrsta["open time"]
        zapremo = vrsta["close time"]
        stolpec_open = f"{dan} open"
        stolpec_close = f"{dan} close"
        hours_spremenjeno.loc[business_id][stolpec_open] = odpremo
        hours_spremenjeno.loc[business_id][stolpec_close] = zapremo
    return np.array(hours_spremenjeno)


def razsiri_z_attributes(business_ids):
    """
    Naredimo veliko novih stolpcev: za vsak atribut posebej, vsi so nominalni
    """
    neumna_vrednost = "-99999"
    tabela_atributov = PODATKI["attributes"]
    imena_atributov = list(tabela_atributov["name"].unique())
    nova_tabela = pd.DataFrame(index=business_ids, columns=imena_atributov, data=neumna_vrednost)
    for _, vrsta in tabela_atributov.iterrows():
        business_id = vrsta["business_id"]
        atribut = vrsta["name"]
        vrednost = vrsta["value"]
        nova_tabela.loc[business_id][atribut] = vrednost
    for stolpec in imena_atributov:
        numericne_vrednosti = pd.to_numeric(nova_tabela[stolpec], errors='coerce')
        if all(numericne_vrednosti.notnull()):
            nova_tabela[stolpec] = numericne_vrednosti
    # pri spodnjem klicu bomo odvrgli stolpec, ki pripada neumni vrednosti
    return pretvori_vse_one_hot(
        nova_tabela,
        list(nova_tabela.columns),
        drop=[neumna_vrednost]
    )


# Razširitev osnovne tabele s tabelo _REVIEW_

Iz tabele `review` izluščite zgolj stolpce, ki jih potrebujete:
  - `business_id` (ta povezuje tabeli `business` in `review`)
  - koristni stolpci, ki jih bomo uporabili za učenje

Izračunajte vrednosti atributov v novi tabeli tako, da uporabite
pandasovi metodi `groupby(seznam_stolpcev)` in `agg(seznam_agregatorejv)`:
  - Ti dve sta zelo pogosto uporabljeni _v verigi_ ena za drugo:
    `nova = podatki.groupby(...).agg(...)`

Če preverimo imena stolpcev, vidimo, da niso nizi, ampak pari, npr.
`("stars", "max")`. Spremenite jih v nize. Spomnimo se:
  - `"abc".join(("AA", "BB", "CC"))` vrne niz `AAabcBBabcCC`
  - imena stolpcev so shranjena v `tabela.columns`. Posodobimo jih
    lahko s klicem `tabela.columns = ...`

Uporaba `groupby` in `agg` je povzročila, da je sedaj `business_id`
indeks novonastale tabele (preverimo!), a nič name ne zagotavlja, da
  - je vrstni red vrstic v njej isti kot v osnovni tabeli,
  - so v novonastali tabeli prisotni čisto vsi poslovni obrati.
Na novonastali tabeli uporabi metodo `reindex(seznam id-jev, fill_value=...)`,
ki bo premešala obstoječe vrstice in dodala tiste, ki manjkajo.

Končno spremenite tabelo v `np.array` z imenom `xs_review`.


In [30]:

koristni_stolpci = ['business_id','stars','cool','funny','useful']
skrceni_review = PODATKI["review"][koristni_stolpci]



nova_tabela = skrceni_review.groupby(['business_id']).agg(['min', 'max', 'sum', 'mean', 'size'])
#nova_tabela.columns = ['_'.join(stolpec) for stolpec in nova_tabela.columns]
nova_tabela.columns = nova_tabela.columns.map("_".join)
nova_tabela = nova_tabela.reindex(biznisi, fill_value=-1)

xs_review = np.array(nova_tabela)


# Razširitev osnovne tabele s tabelo _USERS_

### Predpriprava

Iz tabele `users` smo že izluščili stolpce, ki jih potrebujemo:
  - `user_id` (omogoča povezavo do tabele `review`)
  - numerični stolpci v tabeli
  - `average_rate` (edini kategoričen, ki je tudi uporaben)

Podobno smo storili s tabelo `review`, kjer smo obdržali le
  - `business_id` (povezava s tabelo `business`)
  - `user_id` (povezava s tabelo `users`)

### Naloga

Uporabite pandasovo metodo `pd.merge(leva tabela, desna tabela, on=ime stolpca)` ([dokumentacija](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html))
in združite skrčeni tabeli (s stolpci, opisanimi zgoraj) `users` in `review` v novo.
V novi tabeli imamo
  - vrstice, ki pripadajo vrednostim stolpca `user_id`, ki so prisotne v obeh tabelah,
  - unijo stolpcev obeh tabel (`user_id` se ne ponovi).

Popravite vrednosti stolpca `average_rate` tako, da bo postal numeričen.

V novi tabeli obdržimo le numerične stolpce in `business_id`. Uporabna je metoda
`tabela.select_dtypes(include=["number"])`.

Sedaj smo problem prevedli na prejšnjo nalogo in moramo samo še uporabiti verigo
`groupby`, `agg` in `reindex` (ter na koncu `np.array`).



In [52]:
uporabni_stolpci = ["user_id", "average_rate"]
users: pd.DataFrame = PODATKI["users"]
for stolpec in users.columns:
    if all(pd.to_numeric(users[stolpec], errors='coerce').notnull()):
        uporabni_stolpci.append(stolpec)
users_skrceno = users[uporabni_stolpci]

review_skrceno: pd.DataFrame = PODATKI["review"][["business_id", "user_id"]]

zdruzeno = pd.merge(review_skrceno, users_skrceno, on='user_id')
zdruzeno['average_rate'] = zdruzeno["average_rate"].apply(lambda r: 1.0 if r == 'high' else 0.0)

uporabni_stolpci = ['business_id'] + list(zdruzeno.select_dtypes(include=['number']).columns)
nova_tabela = zdruzeno[uporabni_stolpci]

nova_tabela = nova_tabela\
    .groupby(['business_id'])\
    .agg(['mean', 'min', 'max', 'sum', 'size'])\
    .reindex(biznisi, fill_value=-1)

xs_users = np.array(nova_tabela)


# Preizkušanje razširitev

In [53]:
def preizkusi_vse_moznosti(vsi_bloki: List[np.ndarray], imena_blokov: List[str]):
    n_blokov = len(imena_blokov)
    n_poskusov = 2 ** n_blokov - 1
    rezultati = pd.DataFrame(
        index=range(n_poskusov),
        columns=imena_blokov,
        data=False
    )
    rezultati["Naključni gozd"] = np.zeros(n_poskusov)
    rezultati["Bučman"] = np.zeros(n_poskusov)
    for i, podseznam in enumerate(generiraj_vse_neprazne_podsezname(n_blokov)):
        xs = np.block([vsi_bloki[j] for j in podseznam])
        for ime, tocnost in preizkusi_podatke(xs, y_business).items():
            rezultati.at[i, ime] = tocnost
        for j in podseznam:
            rezultati.at[i, imena_blokov[j]] = True
    rezultati.sort_values(by="Naključni gozd", ascending=False, inplace=True)
    return rezultati


xs_hours = razsiri_z_hours(biznisi)
xs_attributes = razsiri_z_attributes(biznisi)
vsi_bloki = [xs_business, xs_hours, xs_attributes, xs_review, xs_users]
imena_blokov = ["business", "hours", "attributes", "review", "users"]

In [54]:
preizkusi_vse_moznosti(vsi_bloki, imena_blokov)

,business,hours,attributes,review,users,Naključni gozd,Bučman
9,False,True,True,False,False,0.924051,0.310127
2,False,False,True,False,False,0.917722,0.310127
6,True,False,True,False,False,0.917722,0.310127
15,True,True,True,False,False,0.911392,0.310127
12,False,False,True,True,False,0.905063,0.310127
29,False,True,True,True,True,0.892405,0.310127
25,True,True,True,True,False,0.892405,0.310127
22,False,True,True,False,True,0.886076,0.310127
18,True,False,True,True,False,0.879747,0.310127
13,False,False,True,False,True,0.879747,0.310127
